# TD1 & TD2 : Echantillonnage d un signal rectangulaire

Notebook interactif pour comparer le signal continu $x(t)$, son echantillonnage, le signal discret $x[n]$ et la TFD.

- Bleu : signal continu et transformee associee.
- Vert : signal echantillonne et spectre periodise.
- Orange : signal discret $x[n]$ et TFSD (DTFT) quasi-continue.
- Rouge : version zero-padding et TFD sur $K$ points.

Les sliders $T$ et $N$ sont relies par $T = N / F_e$ (ici $F_e$ est fixe). $K$ reste reglable entre $N$ et $3N$ et influe sur la finesse de la TFD.


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

A = 1.0
Fe = 25.0  # Hz (fixe pour tout le notebook)
oversample = 100  # densification temporelle pour un trace lisse
freq_points = 2001  # densite du trace frequentiel
replica_count = 25  # nombre de periodes de spectre a sommer de chaque cote

plt.rcParams["figure.figsize"] = (10, 8)
plt.rcParams["axes.grid"] = True

In [7]:
# Sliders synchronises : T = N / Fe
T_min = 4 / Fe
T_max = 5.0
T0 = 2

T_slider = widgets.FloatSlider(
    value=T0,
    min=T_min,
    max=T_max,
    step=1/Fe,
    description="T (s)",
    readout_format=".4f",
    continuous_update=False,
)
N_slider = widgets.IntSlider(
    value=int(round(T_slider.value * Fe)),
    min=4,
    max=int(T_max * Fe),
    step=1,
    description="N (points=A)",
    continuous_update=False,
)
K_slider = widgets.IntSlider(
    value=N_slider.value,
    min=N_slider.value,
    max=3 * N_slider.value,
    step=1,
    description="K (DFT points)",
    continuous_update=False,
)

show_continuous = widgets.Checkbox(value=True, description="TFSC (signal x(t) à t continu)")
show_sampled = widgets.Checkbox(value=True, description="TFSC (signal échantillonné xe(t))")
show_dtft = widgets.Checkbox(value=True, description="TFSD (signal numérique x[n])")
show_dft = widgets.Checkbox(value=True, description="TFD (signal numérique x[n])")

sync_state = {"lock": False}

def _apply_consistency(new_N: int, keep_k: bool = False):
    new_N = int(max(N_slider.min, min(new_N, N_slider.max)))
    new_T = new_N / Fe
    new_min = new_N
    new_max = 3 * new_N
    if keep_k:
        new_k = int(max(new_min, min(K_slider.value, new_max)))
    else:
        new_k = new_min  # par defaut, K = N quand T ou N changent
    sync_state["lock"] = True
    try:
        K_slider.min = new_min
        K_slider.max = new_max
        K_slider.value = new_k
        N_slider.value = new_N
        T_slider.value = new_T
    finally:
        sync_state["lock"] = False


def _sync_from_T(change):
    if change.get("name") != "value" or sync_state["lock"]:
        return
    new_N = int(round(change["new"] * Fe))
    _apply_consistency(new_N, keep_k=False)


def _sync_from_N(change):
    if change.get("name") != "value" or sync_state["lock"]:
        return
    _apply_consistency(change["new"], keep_k=False)


def _sync_from_K(change):
    if change.get("name") != "value" or sync_state["lock"]:
        return
    _apply_consistency(N_slider.value, keep_k=True)

T_slider.observe(_sync_from_T, names="value")
N_slider.observe(_sync_from_N, names="value")
K_slider.observe(_sync_from_K, names="value")


In [8]:
def rectangular_pulse(t, T, amplitude):
    return amplitude * ((t >= 0) & (t < T)).astype(float)

def continuous_spectrum(f, T, amplitude):
    return amplitude * T * np.sinc(f * T) * np.exp(-1j * np.pi * f * T)

def sampled_spectrum(f, T, amplitude, Te, count):
    acc = np.zeros_like(f, dtype=complex)
    Fe_local = 1.0 / Te
    for k in range(-count, count + 1):
        acc += continuous_spectrum(f - k * Fe_local, T, amplitude)
    return acc / Te

def dtft_spectrum(f, N, amplitude, Fe_local):
    n = np.arange(N)
    expo = np.exp(-1j * 2.0 * np.pi * np.outer(n, f) / Fe_local)
    return amplitude * expo.sum(axis=0)

def dft_spectrum(x, Te):
    X = np.fft.fft(x)
    freqs = np.fft.fftfreq(x.size, d=Te)
    return np.fft.fftshift(freqs), np.fft.fftshift(X)


In [9]:
def plot_signals(T, N, K, show_continuous, show_sampled, show_dtft, show_dft):
    Te = 1.0 / Fe
    T = N / Fe  # garantie de coherence

    display_window = 5.0

    t_dense = np.linspace(0.0, display_window, int(display_window * Fe * oversample) + 1)
    x_cont = rectangular_pulse(t_dense, T, A)

    sample_times = np.arange(0.0, display_window + 1e-12, Te)
    x_samples = rectangular_pulse(sample_times, T, A)

    n_full = np.arange(int(np.ceil(display_window / Te)))
    x_n = A * (n_full < N).astype(float)

    x_dft_time = np.zeros(K)
    fill = min(N, K)
    x_dft_time[:fill] = A

    freq_grid = np.linspace(-Fe, Fe, freq_points)
    X_cont = continuous_spectrum(freq_grid, T, A)
    X_samp = sampled_spectrum(freq_grid, T, A, Te, replica_count)
    X_dtft = dtft_spectrum(freq_grid, N, A, Fe)
    freqs_dft, X_dft = dft_spectrum(x_dft_time, Te)

    zero_idx = int(np.argmin(np.abs(freq_grid)))
    X_cont0 = np.abs(X_cont[zero_idx]) or 1.0
    X_samp0 = np.abs(X_samp[zero_idx]) or 1.0
    X_dtft0 = np.abs(X_dtft[zero_idx]) or 1.0
    dft_zero_idx = int(np.argmin(np.abs(freqs_dft))) if X_dft.size else 0
    X_dft0 = np.abs(X_dft[dft_zero_idx]) if X_dft.size else 1.0
    X_cont_n = X_cont / X_cont0
    X_samp_n = X_samp / X_samp0
    X_dtft_n = X_dtft / X_dtft0
    X_dft_n = X_dft / (X_dft0 if X_dft0 != 0 else 1.0)

    fig, (ax_t, ax_f, ax_f_zoom) = plt.subplots(3, 1, figsize=(10, 10))

    if show_continuous:
        ax_t.plot(t_dense, x_cont, color="C0", label=r"x(t)")

    if show_sampled:
        ax_t.stem(
            sample_times,
            x_samples,
            linefmt="C2-",
            markerfmt="C2^",
            basefmt="gray",
            label=r"x_e(t)",
        )

    if show_dtft:
        ax_t.stem(
            n_full * Te,
            x_n,
            linefmt="C1-",
            markerfmt="C1s",
            basefmt="gray",
            label=r"x[n]",
        )

    if show_dft:
        ax_t.stem(
            np.arange(K) * Te,
            x_dft_time,
            linefmt="C3-",
            markerfmt="C3o",
            basefmt="gray",
            label=r"x[n]",
        )

    ax_t.set_xlim(0.0, display_window)
    ax_t.set_ylim(-0.1, A * 1.3)
    ax_t.set_xlabel("Temps t (s) ou n*Te")
    ax_t.set_ylabel("Amplitude")
    if ax_t.get_legend_handles_labels()[0]:
        ax_t.legend(loc="upper right")

    if show_continuous:
        ax_f.plot(freq_grid, np.abs(X_cont_n), color="C0", label="TF continu")
    if show_sampled:
        ax_f.plot(freq_grid, np.abs(X_samp_n), color="C2", label="TF echantillonne")
    if show_dtft:
        ax_f.plot(freq_grid, np.abs(X_dtft_n), color="C1", label="TFSD (DTFT)")
    if show_dft:
        mask = np.abs(freqs_dft) <= Fe
        ax_f.stem(
            freqs_dft[mask],
            np.abs(X_dft_n)[mask],
            linefmt="C3-",
            markerfmt="C3o",
            basefmt="gray",
            label="TFD",
        )

    ax_f.axvline(-Fe / 2, color="gray", linestyle="--", linewidth=1)
    ax_f.axvline(Fe / 2, color="gray", linestyle="--", linewidth=1)
    ax_f.set_xlim(-Fe, Fe)
    ax_f.set_xlabel("Frequence (Hz)")
    ax_f.set_ylabel("|X(f)| (normalise)")
    if ax_f.get_legend_handles_labels()[0]:
        ax_f.legend(loc="upper right")

    # Zoom frequentiel autour de 0 Hz
    if show_continuous:
        ax_f_zoom.plot(freq_grid, np.abs(X_cont_n), color="C0", label=r"X(f)")
    if show_sampled:
        ax_f_zoom.plot(freq_grid, np.abs(X_samp_n), color="C2", label=r"X_e(f)")
    if show_dtft:
        ax_f_zoom.plot(freq_grid, np.abs(X_dtft_n), color="C1", label=r"X(e^{j2\pi fT_e})")
    if show_dft:
        mask_zoom = (freqs_dft >= -Fe) & (freqs_dft <= Fe)
        ax_f_zoom.stem(
            freqs_dft[mask_zoom],
            np.abs(X_dft_n)[mask_zoom],
            linefmt="C3-",
            markerfmt="C3o",
            basefmt="gray",
            label="X[k]",
        )
    ax_f_zoom.axvline(-Fe / 2, color="gray", linestyle="--", linewidth=1)
    ax_f_zoom.axvline(Fe / 2, color="gray", linestyle="--", linewidth=1)
    ax_f_zoom.set_xlim(-4, 4)
    ax_f_zoom.set_xlabel("Frequence (Hz) - zoom")
    ax_f_zoom.set_ylabel("|X(f)| (normalise)")
    if ax_f_zoom.get_legend_handles_labels()[0]:
        ax_f_zoom.legend(loc="upper right")

    fig.tight_layout()
    plt.show()


In [10]:
controls1 = widgets.VBox(
    [
        widgets.HTML(
            "<b>Reglages</b>"
            # "<br/>T et N restent lies (T = N / F<sub>e</sub>),<br/>"
            # "K ajuste la resolution de la TFD."
        ),
        T_slider,
        N_slider,
        K_slider,
    ],
    layout=widgets.Layout(width="320px"),
)

controls2 = widgets.VBox(
    [
        widgets.HTML("<b>Courbes a afficher</b>"),
        show_continuous,
        show_sampled,
        show_dtft,
        show_dft,
    ],
    layout=widgets.Layout(width="320px"),
)

controls = widgets.HBox([controls1, controls2])

output = widgets.interactive_output(
    plot_signals,
    {
        "T": T_slider,
        "N": N_slider,
        "K": K_slider,
        "show_continuous": show_continuous,
        "show_sampled": show_sampled,
        "show_dtft": show_dtft,
        "show_dft": show_dft,
    },
)

display(widgets.VBox([controls, output]))
